In [43]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome(ChromeDriverManager().install())

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [44]:
produto = 'iphone 12 64 gb'
termos_banidos = '11 13 14 mini watch'

preco_minimo = 3000
preco_maximo = 3500

In [45]:
def busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo):

    produto.lower()
    termos_banidos.lower()

    lista_nomes_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")

    lista_itens_encontrados = []

    driver.get("https://www.google.com.br/")

    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')

    for elemento in elementos:
        if "Shopping" in elemento.text:
            elemento.click()
            break

    lista_resultados = driver.find_elements(By.CLASS_NAME, 'i0X6df')

    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text.lower()

        tem_termos_banidos = False
        for termo in lista_termos_banidos:
            if termo in nome:
                tem_termos_banidos = True

        tem_lista_nomes_produto = True
        for termo in lista_nomes_produto:
            if termo not in nome:
                tem_lista_nomes_produto = False

        if tem_termos_banidos == False and tem_lista_nomes_produto:
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)

            preco_minimo = float(preco_minimo)
            preco_maximo = float(preco_maximo)

            if preco_minimo <= preco <= preco_maximo:

                child = resultado.find_element(By.CLASS_NAME, 'XrAfOe')
                parent = child.find_element(By.XPATH, '..')
                link = parent.get_attribute('href')

                lista_itens_encontrados.append((nome, preco, link))
    return lista_itens_encontrados



    def busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo):
        

In [46]:
lista_ofertas_google_shopping = busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo)
print(lista_ofertas_google_shopping)

[('iphone 12 64gb roxo vitrine', 3300.0, 'https://www.google.com.br/url?url=https://loja.switchphone.com.br/iphone-12-64gb-roxo-vitrine%3Fsrsltid%3DAd5pg_HctguCm9bQvWxqgURp-IQqQ7843JGWLD1JnHpqmJp4gejEMfSaAtE&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwj1_qHOquP8AhXOqpUCHennAXsQgOUECN8L&usg=AOvVaw2119fnUn1eIS3PXRnBr6Oq'), ('iphone 12 64gb preto vitrine', 3300.0, 'https://www.google.com.br/url?url=https://loja.switchphone.com.br/iphone-12-64gb-preto-vitrine%3Fsrsltid%3DAd5pg_EMJ6TiUf2pjqO1Y6s6VqR05JSjhP93zTYLUaIOXDTm4aXFOWqpKBQ&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwj1_qHOquP8AhXOqpUCHennAXsQgOUECJ0M&usg=AOvVaw1FKFQUt0OXgKJ-U4QC_rqx'), ('iphone 12 64gb - azul - estou zerado', 3329.1, 'https://www.google.com.br/url?url=https://www.trocafy.com.br/iphone-12-64gb-azul-sou-como-novo-2552/p%3Fidsku%3D966%26srsltid%3DAd5pg_Htvam3hw9v3vHM_GxsrHNQnqfDmp5mQwONTaOXvY9bHjGLM7EPq2o&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwj1_qHOquP8AhXOqpUCHennAXsQguUECK0M&usg=AOvVaw2T4z8uTvBkB6_WB7gHKZVt'), ('iphone 12 64gb azul vitrine', 3300